<a href="https://colab.research.google.com/github/fpgmina/DeepNLP/blob/main/L3_Part_2_NER_and_Intent_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Deep Natural Language Processing @ PoliTO**

---


**Teaching Assistant:** Giuseppe Gallipoli

**Credits:** Moreno La Quatra

**Practice 3:** Named Entity Recognition (part 1) & Intent Detection (part 2)

## Intent Detection

In data mining, intention mining or intent mining is the problem of determining a user's intention from logs of his/her behavior in interaction with a computer system, such as in search engines. Intent Detection is the identification and categorization of what a user online intended or wanted to find when they type or speak with a conversational agent (or a search engine).

![https://d33wubrfki0l68.cloudfront.net/32e2326762c75a0357ab1ae1976a60d4bbce724b/f4ac0/static/a5878ba6b0e4e77163dc07d07ecf2291/2b6c7/intent-classification-normal.png](https://d33wubrfki0l68.cloudfront.net/32e2326762c75a0357ab1ae1976a60d4bbce724b/f4ac0/static/a5878ba6b0e4e77163dc07d07ecf2291/2b6c7/intent-classification-normal.png)

In this section, you will use the ATIS dataset: https://github.com/yvchen/JointSLU; https://www.kaggle.com/siddhadev/atis-dataset-clean/home

The task is to classify the intent of a sentence. The dataset is split into train, validation and test sets. **Use the provided splits** to train and evaluate your models.

In [ ]:
!wget https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P4/IntentDetection/atis.train.csv
!wget https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P4/IntentDetection/atis.dev.csv
!wget https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P4/IntentDetection/atis.test.csv

### **Question 5: Two-step classification model**

Train a classification model to identify the intent from a given sentence. The model is required to leverage on pre-trained BERT model to generate sentence embeddings (important: **no fine-tuning**). The model is required to use the embeddings to perform classification.

Once extracted the embeddings, you can use any classifier you want. For example, you can use a linear classifier (e.g., Logistic Regression) or a neural network (e.g., MLP). For your convenience, you can use the `sklearn` library for training the classifier (https://scikit-learn.org/stable/supervised_learning.html).

![https://github.com/MorenoLaQuatra/DeepNLP/blob/main/practices/P4/IntentDetection/no_finetuning.png?raw=true](https://github.com/MorenoLaQuatra/DeepNLP/blob/main/practices/P4/IntentDetection/no_finetuning.png?raw=true)


Assess the performance of the trained model (the model on top of BERT) on the test set by using the **classification accuracy**, **precision**, **recall** and **F1-score**. You can use the `sklearn` library for computing the metrics (https://scikit-learn.org/stable/api/sklearn.metrics.html).


Note: you can use the `sentence-transformers` library to generate sentence embeddings (https://www.sbert.net/docs/pretrained_models.html).

In [ ]:
%%capture
!pip install sentence-transformers
!pip install sklearn

In [ ]:
# your code here

### **Question 6: Fine-tuning end-to-end classification model**

Another approach is to fine-tune the BERT model for the classification task. A classification head is added on top of the pre-trained BERT model. The classification head is trained end-to-end with the BERT model.
This approach is more effective than the previous one because the model is trained end-to-end. However, the model requires more training time and resources.

Train a new BERT model for the task of [sequence classification](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForSequenceClassification) (include BERT fine-tuning).  

![https://github.com/MorenoLaQuatra/DeepNLP/blob/main/practices/P4/IntentDetection/finetuning.png?raw=true](https://github.com/MorenoLaQuatra/DeepNLP/blob/main/practices/P4/IntentDetection/finetuning.png?raw=true)

Assess the performance of the generated model by using the same metrics used in the previous question.

Which model has better performance? Why?

In [ ]:
# your code here

### **Question 7: Intent Detection using Large Language Models**
**Credits:** Giuseppe Gallipoli

#### Introduction
[Large Language Models](https://en.wikipedia.org/wiki/Large_language_model) (LLMs) are a type of deep learning model capable of language generation. These models are built on deep learning architectures, primarily using neural networks, and are trained on massive amounts of text data. LLMs generally leverage the *Transformer* architecture, which allows them to process language in context, capturing complex relationships between words and concepts.

Large Language Models have demonstrated excellent capabilities across a wide variety of tasks, making them versatile models which can be applied in diverse scenarios and use cases. Although they are more typically used for *generative* tasks (e.g., text generation, text summarization, open-ended Question Answering), they can also be employed in *discriminative* tasks.

In this practice, we will use a Large Language Model to address an intent detection task. Rather than using a pre-trained encoder-only model without (Question 5) or with fine-tuning (Question 6), we will ask the LLM to classify the intent given a sentence of interest.
<br><br>
For now, we will use the [Phi-4-mini-instruct](https://huggingface.co/microsoft/Phi-4-mini-instruct) 3.8B model, i.e., `microsoft/Phi-4-mini-instruct`.

#### LLM Prompting
To interact with LLMs, we need to define a **prompt**, which is a piece of text containing the instruction or question we want to give or ask the model. As we saw in Practice 1 (Exercise 11), a prompt can both include only the instruction/question for the LLM but also additional information (i.e., *context*) which can be exploited by the model to generate the answer.

For now, we will ask the model to classify the intent of a given sentence <u>without providing</u> any additional context. Please note that, since we want to (potentially) limit the choice of the LLM to a predefined list of intents (i.e., the set of labels of our dataset), we will also provide this list to the model.\
*Example of prompt*:\
Which is the intent of the following sentence?\
Choose among: label1, label2, ...
<br><br>

**Hint**: For those data instances whose intent is the concatenation of two intents (i.e., `atis_flight#atis_airfare`), consider only the first one.

<u>Suggestion</u>: To increase speed, switch to a GPU runtime. You can do this by clicking on Runtime → Change runtime type → Hardware accelerator → Select T4 GPU.\
If you encounter an `OutOfMemoryError`, try restarting the session by clicking on Runtime → Restart session.

In [ ]:
# your code here

#### Few-Shot Learning
When we provide the LLM with additional context to be leveraged for generating an answer, this is known as *in-context learning* (ICL).\
A common technique to perform ICL involves including one or more **additional examples** of questions (or instructions) and their expected answers in the prompt.\
This can be useful for several reasons: telling the model the required output format, both in terms of structure and style, making the model reason about the provided examples to better adapt to the (new) task, tailoring model's responses to a user's specific needs, ...\
All of this is done directly at inference, without the need of further training the model, simply by leveraging the reasoning and generalization capabilities of LLMs.\
When no examples are provided, as in the previous point of the exercise, we talk about **zero-shot** learning. Conversely, when we supply additional input-output examples in the model's prompt, we talk about **few-shot** learning or $k$-shot learning, where $k$ represents the number of ICL examples.\
Examples can be selected according to different strategies, either randomly or in a more clever way, but please note that they <u>must be chosen from the training set</u> to avoid *data leakage*.
<br><br>

Now, try to implement few-shot learning by modifying the previous point of the exercise so that the prompt can include $k$ additional examples.

In [ ]:
# your code here

Try with different $k$ values, e.g., $k \in [1, 3, 5, 10]$, and see if and how the model's performance changes.

In [ ]:
# your code here

Until now, we have used the [Phi-4-mini-instruct](https://huggingface.co/microsoft/Phi-4-mini-instruct) 3.8B model, i.e., `microsoft/Phi-4-mini-instruct`, but there are plenty of LLMs!\
Try another model of your choice, e.g., [Mistral-Instruct](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3) 7B `mistralai/Mistral-7B-Instruct-v0.3`. Remember to always keep in mind the GPU memory constraints you have.
<br><br>

**Hint**: To download and use certain models, it may be needed to request access to them (e.g., [Llama3.2-Instruct](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct) 3B). This can be done from the Hugging Face model page after logging in. Once access is granted, you need to create your personal access token (go to your HF profile, Settings → Access Tokens → Create new token) and authenticate using it when downloading both the model and the tokenizer.\
To authenticate, you can either use the additional `token` parameter in the `from_pretrained()` method or via the [HF Command Line Interface](https://huggingface.co/docs/huggingface_hub/guides/cli).
```
model_tag = 'MODEL_TAG'
HF_TOKEN = 'MY_HF_TOKEN'

model = AutoModelForCausalLM.from_pretrained(model_tag, torch_dtype=torch.float16, token=HF_TOKEN, device_map=device)
tokenizer = AutoTokenizer.from_pretrained(model_tag, token=HF_TOKEN)
```

In [ ]:
# your code here

After analyzing the performance of the different settings and/or models, here you can find some questions to reason about the results:
- What is the performance in the zero-shot learning setting? (i.e., $k=0$)
- How does it change when providing one additional example? (i.e., $k=1$)
- What happens when increasing the number of ICL examples?
- If you tested additional models, which one performs best in the zero-shot and few-shot learning settings?
- What challenges or limitations did you observe?